In [8]:
import pandas as pd
from cyclum.hdfrw import hdf2mat
import numpy as np
from math import pi

pseudotime = hdf2mat("/home/shaoheng/Documents/data/McDavid/pc3-pseudotime.h5").values
cpt = pd.read_pickle('/home/shaoheng/Documents/data/McDavid/pc3_cpt.pkl').values


In [47]:
def empirical_precision(pseudotime, label, granularity: int = 100):
    
    pseudotime = np.squeeze(np.array(pseudotime))
    pseudotime = pseudotime % (2 * pi)
    
    n = label.shape[0]
    
    label = np.squeeze(np.array(label))
    unique_label = np.unique(label)
    
    onehot_label = np.repeat(unique_label.reshape([-1, 1]), n, axis=1) == label
    splits = np.linspace(0, 2 * pi, granularity)
    
    # 0---i-----j-----k---2pi:
    # [k,2pi)U[0, i), [i, j), [j, k)

    p = 0
    
    for i in range(granularity):  # 0 <= i < granularity
        for j in range(i, granularity):  # i <= j < granularity
            for k in range(j, granularity):  # j <= k < granularity
                x = (pseudotime < splits[i]) | (pseudotime >= splits[k])
                y = (pseudotime >= splits[i]) & (pseudotime < splits[j])
                z = (pseudotime >= splits[j]) & (pseudotime < splits[k])
                
                r = np.vstack([x, y, z])
                
                p = max(p, np.sum(onehot_label[[0, 1, 2], :] & r) / n)
                p = max(p, np.sum(onehot_label[[1, 2, 0], :] & r) / n)
                p = max(p, np.sum(onehot_label[[2, 0, 1], :] & r) / n)
                p = max(p, np.sum(onehot_label[[2, 1, 0], :] & r) / n)
                p = max(p, np.sum(onehot_label[[1, 0, 2], :] & r) / n)
                p = max(p, np.sum(onehot_label[[0, 2, 1], :] & r) / n)
    return p  
 
empirical_precision(pseudotime, cpt)


0.6204986149584487